In [2]:
# import transformers
import torch
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer

In [2]:
# sentiment_pipeline = pipeline("sentiment-analysis")
# data = ["I love you", "I hate you", "Dogs are awesome"]
# sentiment_pipeline(data)

In [11]:
from transformers import pipeline

specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
data = ["I love you", "I hate you"]
specific_model(data)

ImportError: cannot import name 'pipeline' from 'transformers' (unknown location)

In [217]:
# specific_model('awesome perfect very good serial killer maniac')
specific_model("Life’s good, you should get one.")

[{'label': 'POS', 'score': 0.9907208681106567}]

In [149]:
lst = (('perfect', 'awesome', 'brilliant fascinating charming very good the best'), ('killer', 'bad', 'idiot'))
sentense = [[], []]
result = {}
idx = 0
while idx < 3:
    sentense[1].append(lst[1][idx])
    joined_sent = ' '.join(sentense[0] + sentense[1])
    result[joined_sent] = specific_model(joined_sent)[0]['label'] + " " + str(specific_model(joined_sent)[0]['score'])
    sentense[0].append(lst[0][idx])
    joined_sent = ' '.join(sentense[0] + sentense[1])
    result[joined_sent] = specific_model(joined_sent)[0]['label'] + " " + str(specific_model(joined_sent)[0]['score'])
    idx += 1
print(result)

{'killer': 'NEG 0.8677386045455933', 'perfect killer': 'POS 0.6435449123382568', 'perfect killer bad': 'NEG 0.9420040249824524', 'perfect awesome killer bad': 'POS 0.5955454707145691', 'perfect awesome killer bad idiot': 'NEG 0.9756888747215271', 'perfect awesome brilliant fascinating charming very good the best killer bad idiot': 'POS 0.9804194569587708'}


In [5]:
imdb = load_dataset("imdb")

Found cached dataset imdb (/Users/georgekokkin/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [42]:
imdb = load_dataset("imdb")
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])
# print(small_train_dataset['text'][2999])

Loading cached shuffled indices for dataset at /Users/georgekokkin/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-8a9e43a6ac4acdff.arrow
Loading cached shuffled indices for dataset at /Users/georgekokkin/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2eff9f118d84c6fe.arrow


In [43]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /Users/georgekokkin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/georgekokkin/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/043235d6088ecd3dd5fb5ca3592b6913fd516027/vocab.txt
loading file tokenizer.json from cache at /Users/georgekokkin/.cache/huggingface/

In [47]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
 
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /Users/georgekokkin/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-0b7e3e988817bdca.arrow
Loading cached processed dataset at /Users/georgekokkin/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-68a5db7b1564ae8f.arrow


In [48]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [218]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [50]:
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [51]:
training_args = TrainingArguments(
   output_dir='models',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch"
#    push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/georgekokkin/Documents/Python/Hillel-ML/venv_ml/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 376
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokeni

Step,Training Loss


Saving model checkpoint to models/checkpoint-188
Configuration saved in models/checkpoint-188/config.json
Model weights saved in models/checkpoint-188/pytorch_model.bin
tokenizer config file saved in models/checkpoint-188/tokenizer_config.json
Special tokens file saved in models/checkpoint-188/special_tokens_map.json
Saving model checkpoint to models/checkpoint-376
Configuration saved in models/checkpoint-376/config.json
Model weights saved in models/checkpoint-376/pytorch_model.bin
tokenizer config file saved in models/checkpoint-376/tokenizer_config.json
Special tokens file saved in models/checkpoint-376/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=376, training_loss=0.2881764959781728, metrics={'train_runtime': 17256.6163, 'train_samples_per_second': 0.348, 'train_steps_per_second': 0.022, 'total_flos': 783875831546880.0, 'train_loss': 0.2881764959781728, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('models/checkpoint-376')
model = AutoModelForSequenceClassification.from_pretrained('models/checkpoint-376')


In [11]:
# print(dir(model))

In [8]:
from transformers import pipeline

In [26]:
sentiment_model = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
print(sentiment_model(['Well. What did I just watch ? I\'m not sure if I can find words to describe this movie.']))

[{'label': 'LABEL_0', 'score': 0.9316208958625793}]
